In [3]:
import pandas as pd
import numpy as np

In [5]:
mi_df =  pd.read_csv(
    'data/01_data_cruda/MI_data.csv', 
    header=0, 
    sep=","
)

In [6]:
len(mi_df)

1700

In [ ]:
mi_df.columns.tolist()

In [ ]:
mi_df["REC_IM"].unique()